In [2]:
import json

with open('config.json', 'r') as file:
    config_info = json.load(file)

DATA_init = config_info["DATA_init"] 
#DATA_init = '/DATA_init/' #Location where NABirds dataset is located. For instance, "DATA_init+'nabirds/'" should be the path of the NABirds dataset.
FOLDER_init = config_info["FOLDER_init"]
#FOLDER_init = '/FOLDER_init/' #Location where this repogistory "Inspecting_Hierarchies_ML" is located.

DATA_PATH = DATA_init+'nabirds/'

/DATA_init/nabirds/images_2341234/0991


RuntimeError: No active exception to reraise

In [ ]:
import os
try:
    os.mkdir(DATA_PATH+'images_128') 
except:
    pass

cls_nm_list=os.listdir(DATA_PATH+'images')
#Generates folders for 128x128 resized images
for l in cls_nm_list:
    os.mkdir(DATA_PATH+'images_128/'+l) 
    print(DATA_PATH+'images_128/'+l)

In [ ]:
root_list=[]
cls_list=[]
data_info=[]
cnt=0
for root, directories, filenames in os.walk(DATA_PATH): 
    directories.sort()
    chk_root = not (base_PATH+'/_') in root
    chk_jpg = [fnm for fnm in filenames if fnm.split('.j')[-1]=='pg'] #Only check '.jpg' extension files. (It ignores '.JPG' extension files using capital letters)
    chk_nonjpg = [fnm for fnm in filenames if fnm.split('.j')[-1]!='pg']
    #print(root, filenames)
    
    #The following four lines will replace '.JPG' extension files to '.jpg' extension files. 
    if (len(chk_nonjpg)>=1):
        for i in range(len(chk_nonjpg)):
            print(root+'/'+chk_nonjpg[i])
            os.rename(root+'/'+chk_nonjpg[i], root+'/'+chk_nonjpg[i].replace('.JPG','.jpg'))

In [4]:
import torch
import numpy as np
import math
import argparse
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
import sys
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]=str(0) #You may want to change GPU number

dataset='nabirds'

gray = False
    
data_info = pd.read_csv(FOLDER_init+'Inspecting_Hierarchies_ML/nabirds_info.csv')
data_cls = pd.read_csv(FOLDER_init+'Inspecting_Hierarchies_ML/nabirds_cls2.csv',delimiter='|')


N= len(data_info) #Number of images: 48562


N_train = int(0.7*N) #70%
N_val = int(0.1*N) #10%
N_test = N-N_train-N_val #20%

n_classes = len(data_cls) #Number of clasees

resize=128 

pd.options.mode.chained_assignment = None
        
        
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from PIL import Image
import torchvision.transforms.functional as TF



class NabirdsDataset(Dataset):
    def __init__(self, info_path, cls_path, transform=None):
        self.data_info = pd.read_csv(info_path)
        self.data_cls = pd.read_csv(cls_path)
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.data_info.iloc[index,0]
        img_path = img_path.replace('images_128/','images/')
        
        img = Image.open(img_path).convert('RGB')
        cls = self.data_info.iloc[index,1]
        
        label = torch.tensor(int(self.data_cls.loc[self.data_cls['Class']==cls,'Number']))
        if self.transform is not None:
            img = self.transform(img)    
            
        img = TF.resize(img, size=[resize, resize]) #size=[128,128]) 
        img.save(img_path.replace('images/','images_'+str(resize)+'/')) #Save resized images 
        return (img, label)

    def __len__(self):
        return len(self.data_info)

nabirds_dataset = NabirdsDataset(info_path=FOLDER_init+'Inspecting_Hierarchies_ML/'+dataset+'_info.csv', 
                              cls_path=FOLDER_init+'Inspecting_Hierarchies_ML/'+dataset+'_cls.csv',transform=None)
train_set, val_set, test_set = torch.utils.data.random_split(nabirds_dataset, lengths=[N_train, N_val, N_test], generator=torch.Generator().manual_seed(1)) 


class MapDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, map_fn):
        self.dataset = dataset
        self.map = map_fn

    def __getitem__(self, index):
        img, label= self.dataset[index]
        img = self.map(img)
        return (img, label)

    def __len__(self):
        return len(self.dataset)


AUG=False
train_set_tf = MapDataset(train_set, transforms.ToTensor())
val_set_tf = MapDataset(val_set,  transforms.ToTensor())
test_set_tf = MapDataset(test_set,  transforms.ToTensor())


#trainloader = DataLoader(train_set_tf, batch_size=32, shuffle=True)
trainloader2 = DataLoader(MapDataset(train_set, transforms.ToTensor()), batch_size=200, shuffle=True)

#valloader = DataLoader(val_set_tf, batch_size=32)
valloader2 = DataLoader(MapDataset(val_set, transforms.ToTensor()), batch_size=200, shuffle=True) 

#testloader = DataLoader(test_set_tf, batch_size=32)
testloader2 = DataLoader(test_set_tf, batch_size=200, shuffle=True) 


'''Analyze training set'''
use_cuda = torch.cuda.is_available()
labels_np=np.ones(0)*np.nan
t1=time.time()
for i, data in enumerate(trainloader2):
    # get the inputs; data is a list of [inputs, labels]
    if i%10==0:
        t2=time.time()
        print(i, 100*i/len(trainloader2))
        print('Spend time:', t2-t1)
    inputs, labels = data

    if use_cuda:
        inputs, labels = inputs.cuda(), labels.cuda()
    labels_np=np.concatenate([labels_np, labels.cpu().numpy()],axis=0)
        
print('Finished for training data\n\n')
for i, data in enumerate(valloader2):
    # get the inputs; data is a list of [inputs, labels]
    if i%10==0:
        t2=time.time()
        print(i, 100*i/len(valloader2))
        print('Spend time:', t2-t1)
    inputs, labels = data
    
print('Finished for validation data\n\n')
for i, data in enumerate(testloader2):
    # get the inputs; data is a list of [inputs, labels]
    if i%10==0:
        t2=time.time()
        print(i, 100*i/len(testloader2))
        print('Spend time:', t2-t1)
    inputs, labels = data
        
print('Finished for test data\n\n')

cls_unq, cls_cnt= np.unique(labels_np,return_counts=True)
log_cls_prior = np.log(cls_cnt/np.sum(cls_cnt))  #np.log(cls_cnt)-np.mean(np.log(cls_cnt))
inputs.shape, cls_unq, cls_cnt, len(cls_unq), log_cls_prior

FileNotFoundError: [Errno 2] File /FOLDER_init/Inspecting_Hierarchies_ML/nabirds_info.csv does not exist: '/FOLDER_init/Inspecting_Hierarchies_ML/nabirds_info.csv'